In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


/Users/shanekercheval/repos/sik-llms/.venv/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Chat

In [2]:
from sik_llms import create_client, user_message, ChatChunkResponse

model = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously

In [3]:
model(messages=[message])

ChatResponseSummary(content='The capital of France is Paris.', total_input_tokens=14, total_output_tokens=7, total_input_cost=2.1e-06, total_output_cost=4.2e-06, duration_seconds=0.5634920597076416)

### Run Asynchronously

In [4]:
responses = []
async for response in model.run_async(messages=[message]):
    if isinstance(response, ChatChunkResponse):
        print(response.content, end="")
        responses.append(response)

The capital of France is Paris.

In [5]:
print(response)

content='The capital of France is Paris.' total_input_tokens=14 total_output_tokens=7 total_input_cost=2.1e-06 total_output_cost=4.2e-06 duration_seconds=0.4606339931488037


# OpenAI Functions

In [7]:
from sik_llms import Function, Parameter, RegisteredClients

weather_tool = Function(
    name="get_weather",
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name="location",
            type="string",
            required=True,
            description="The city and country for weather info.",
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_FUNCTIONS,
    model_name='gpt-4o-mini',
    functions=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = client(messages=[message])
response

FunctionCallResponse(function_call=FunctionCallResult(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_FwCxd4Wucp2mgWYwVrJ92L6d'), input_tokens=60, output_tokens=17, input_cost=9e-06, output_cost=1.0199999999999999e-05)